In [ ]:
!pip install pytz

In [ ]:
!pip install lxml

In [ ]:
import cchardet

In [ ]:
# LISTS

tester = []
augment_data = {}
existing_game_ids = {}

In [ ]:
import requests
from bs4 import BeautifulSoup
def refreshLeaders():
    top_players = []
    page = 1
    while page < 15:
        base_url = 'https://lolchess.gg/leaderboards?mode=ranked&region=na&page='
        url = base_url + str(page)
        response = requests.get(url)

        # create soup
        soup = BeautifulSoup(response.text, 'lxml')

        # Find top 100 NA 
        elements = soup.find_all('a')

        # extract player links
        for a_element in elements:
            href = a_element.get('href')
            if href and href.startswith('https://lolchess.gg/profile/'):
                text = a_element.text.strip()
                top_players.append(''.join(text.split()))
        page += 1
    return top_players

In [ ]:
from datetime import datetime, timedelta
import pytz
def is_valid_match(timeString):
    patch_datetime = datetime(year=2023, month=8, day=22, hour=19, minute=0, second=0)
    current_time = datetime.utcnow()
    est_tz = pytz.timezone('US/Eastern')
    current_time_est = current_time.replace(tzinfo=pytz.utc).astimezone(est_tz)
    
    timestamp = timeString.split(' ')[0]
    integer = timestamp[:-1]
    unit = timestamp[-1:]
    current_time_est = current_time_est.replace(tzinfo=None)
    match_time = None;
    if unit == 'h':
        match_time = current_time_est - timedelta(hours=int(integer))
    elif unit == 'm' :
        match_time = current_time_est - timedelta(minutes=int(integer))
    elif unit == 's':
        match_time = current_time_est - timedelta(seconds=int(integer))
    elif unit == 'd':
        match_time = current_time_est - timedelta(days=int(integer))
    elif unit == 'w':
        match_time = current_time_est - timedelta(weeks=int(integer))
    return match_time > patch_datetime


In [ ]:
import hashlib
import json
def generate_game_id(game_data):
    game_data_str = json.dumps(game_data, sort_keys=True)

    # generate hash
    sha256_hash = hashlib.sha256(game_data_str.encode()).hexdigest()

    # first 10 of hash as gameid
    game_id = "game_" + sha256_hash[:10]

    # add to game data dict
    game_data["game_id"] = game_id

    return game_id

In [ ]:
config = {
    'user': None,
    'password': None,
    'host': None,
    'database': None,
}

In [ ]:
import mysql.connector
from mysql.connector import pooling

connection_pool = pooling.MySQLConnectionPool(pool_name="game_data_pool",
                                              pool_size=20,
                                              **config)

def connect_to_database():
    try:
        cnx = connection_pool.get_connection()
        print('Connected to Google Cloud MySQL database!')
        return cnx

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        raise


def insert_game_data(cnx, game_data):
    try:
        cursor = cnx.cursor()

        # Insert game data into the 'game' table
        insert_game_query = (
            "INSERT INTO game (game_id, patch_number, placement) "
            "VALUES (%s, %s, %s)"
        )
        game_values = (
            game_data['game_id'],
            game_data['patch_number'],
            game_data['placement']
        )
        cursor.execute(insert_game_query, game_values)

        # Insert game augments 
        insert_augment_query = (
            "INSERT INTO game_augments (game_id, augment_name, augment_stage) "
            "VALUES (%s, %s, %s)"
        )
        augment_data = [(game_data['game_id'], augment_entry['augment_name'][0], augment_entry['augment_name'][1]) for augment_entry in game_data['game_augments']]
        cursor.executemany(insert_augment_query, augment_data)

        # Insert traits 
        insert_trait_query = (
            "INSERT INTO game_traits (game_id, trait_name, trait_level) "
            "VALUES (%s, %s, %s)"
        )
        trait_data = [(game_data['game_id'], trait_entry['trait_name'], trait_entry['trait_level']) for trait_entry in game_data['traits']]
        cursor.executemany(insert_trait_query, trait_data)

        # Insert units data
        insert_unit_query = (
            "INSERT INTO game_units (game_id, unit_name, unit_level) "
            "VALUES (%s, %s, %s)"
        )
        unit_data = [(game_data['game_id'], unit_entry['unit_name'], unit_entry['unit_level']) for unit_entry in game_data['units']]
        cursor.executemany(insert_unit_query, unit_data)

        # Insert unit items
        insert_item_query = (
            "INSERT INTO unit_items (game_id, unit_name, item_name) "
            "VALUES (%s, %s, %s)"
        )
        item_data = [(game_data['game_id'], unit_item['unit_name'], unit_item['item_name']) for unit_item in game_data['unit_items']]
        cursor.executemany(insert_item_query, item_data)

        cursor.close()
        print(".", end="")
        
    except Exception as e:
        print(f"Error: {e}")

def insert_game_entries(cnx, game_entries):
    cursor = cnx.cursor()

    # Insert game entries in a single batch
    for game_data in game_entries:
        insert_game_data(cnx, game_data)

    cursor.close()
    cnx.commit()


In [ ]:
def format_augments(augments):
    # Check if 3 augments
    if len(augments) == 3:
        # create tuples list
        formatted_augments = []
        for i, augment in enumerate(augments, start=2):
            stage = f"{i}-{i-1}"
            formatted_augments.append((augment, stage))
        return formatted_augments
    else:
        return [(augment, None) for augment in augments]



In [ ]:
def get_player_matches(username, cnx):
    print(f'getting data for {username}, ', end="")
    base_url = f'https://lolchess.gg/profile/na/{username}/s9/matches/ranked/'
    continue_searching = True
    page = 1
    game_entries = []  

    while continue_searching:
        url = base_url + str(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        elements = soup.select('[class^="profile__match-history-v2__item placement-"]')
        if len(elements) == 0:
            break
        for element in elements:
            age_element = element.select_one('.age')
            age_text = age_element.text.strip()
            is_valid = is_valid_match(age_text)
            if is_valid:
                # get placement
                score_element = element.find(class_="placement")
                score = score_element.text.strip()[1:]
                
                # get augments
                augments = []
                augment_elements = element.select('.augment')
                for augment_element in augment_elements:
                    image_element = augment_element.find('img')
                    alt_text = image_element.get('alt')
                    augments.append(alt_text)
                
                augments = format_augments(augments)
                # get traits
                traits = []
                traits_elements = element.select('.traits')
                for traits_element in traits_elements:
                    image_elements = traits_element.find_all('img')
                    for image_element in image_elements:
                        trait_text = image_element.get('alt')
                        trait_name = trait_text[2:]
                        trait_level = trait_text[0:1]
                        traits.append([trait_name, trait_level])
                
                # get units and their level and items
                units = []
                units_elements = element.select('.unit')
                for unit_element in units_elements:
                    level_and_unit = unit_element.find_all('img')
                    if len(level_and_unit) == 2:
                        unit_level = level_and_unit[0].get('src')[-5]
                        unit_name = level_and_unit[1].get('alt')
                    else:
                        try:
                            unit_level = level_and_unit[0].get('src')[-5]
                            unit_name = level_and_unit[1].get('alt')
                        except:
                            continue
                    
                    item_elements = unit_element.select('.items')
                    items = []
                    for item_element in item_elements:
                        individual_items = item_element.find_all('img')
                        if individual_items:
                            for item in individual_items: 
                                items.append(item.get('title'))
                    if unit_name:
                        units.append([unit_name, unit_level, items])
                
                # make game_data 
                game_data = {
                    'placement': score,
                    'game_augments': [],
                    'units': [],
                    'unit_items': [],
                    'traits': [],
                    'patch_number': '13.14b'
                }
                
                for augment_entry in augments:
                    game_data["game_augments"].append({"augment_name": augment_entry})
                    
                for trait_entry in traits:
                    trait_name, trait_level = trait_entry
                    game_data["traits"].append({"trait_name": trait_name, "trait_level": int(trait_level)})
                    
                for unit_entry in units:
                    unit_name, unit_level, unit_items = unit_entry

                    # append unit data
                    game_data["units"].append({"unit_name": unit_name, "unit_level": int(unit_level)})

                    # append unit items data
                    unit_items_data = [{"unit_name": unit_name, "item_name": item_name} for item_name in unit_items]
                    game_data["unit_items"].extend(unit_items_data)

                game_id = generate_game_id(game_data)
                
                # check if game already exists in database
                if game_id in existing_game_ids:
                    continue_searching = False
                else:
                    game_entries.append(game_data)
                    existing_game_ids.add(game_id)
            else:
                continue_searching = False
                soup.decompose()
                break
        soup.decompose()
        page += 1

    # insert game entries
    insert_game_entries(cnx, game_entries)


In [ ]:
while True:
    top_players = refreshLeaders()
    cnx = connect_to_database()
    cursor = cnx.cursor()
    query = "SELECT game_id FROM game"
    cursor.execute(query)
    existing_game_ids = {result[0] for result in cursor.fetchall()}
    for player in top_players:
        get_player_matches(player, cnx)
    cnx.close()